In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from LumFunc_miniJPAS import LumFunc
from perturbed_phot import perturb_phot
from my_functions import *
from colorplot import *

In [ ]:
cat = load_noflag_cat('pkl/catalogDual_pz.pkl')

In [ ]:
def load_mags(nb_ind):
#     nb_ind = 11 # J0480
    bb_ind = -3 # g

    mask_fzero = (cat['MAG'][:, nb_ind] < 90) & (cat['MAG'][:, bb_ind] < 90)

    nb_m = cat['MAG'][mask_fzero, nb_ind]
    bb_m = cat['MAG'][mask_fzero, bb_ind]
    nb_e = cat['ERR'][mask_fzero, nb_ind]
    bb_e = cat['ERR'][mask_fzero, bb_ind]

    # Define binning
    m_min = 14
    m_max = 26
    m_bin_n = 75
    x_e = np.linspace(m_min, m_max, m_bin_n)

    # SNR=5 cut
    bbcut = x_e[np.nanargmin(np.abs(m_err_bin(bb_m, bb_e, x_e, bb_m) - 0.24))]
    nbcut = x_e[np.nanargmin(np.abs(m_err_bin(nb_m, nb_e, x_e, nb_m) - 0.24))]
    
    return nb_m, bb_m, nb_e, bb_e, bbcut, nbcut

In [ ]:
nb_ind = 11
bb_ind = -3
nb_m, bb_m, nb_e, bb_e, bbcut, nbcut = load_mags(nb_ind)

n_lm = 50
total_sel = np.zeros(nb_m.shape)

w_pivot = central_wavelength(load_tcurves(load_filter_tags()))[nb_ind]
w_pivot_bb = central_wavelength(load_tcurves(load_filter_tags()))[bb_ind]
w_fwhm = nb_fwhm(load_tcurves(load_filter_tags()), nb_ind, True)
n_bins = 25
L_min = 43.5
L_max = 45

LF_y = np.zeros((n_lm, n_bins))
errors_total = np.zeros((n_lm, n_bins))

for i in range(n_lm):
    print('{}/{}'.format(str(i+1), str(n_lm), end='\r'))
    this_sel, mag_bb, mag_nb = perturb_phot(nb_m, nb_e, bb_m, bb_e,
                                            30, nb_ind, 1, 
                                            bbcut, nbcut)
    idx = np.where(this_sel)
    f_line = mag_to_flux(mag_nb[idx], w_pivot) - mag_to_flux(mag_bb[idx], w_pivot_bb)
    total_sel += this_sel
    x, y, error = LumFunc(f_line, w_pivot, w_fwhm, n_bins, L_min, L_max)
    LF_x = x
    LF_y[i] = y
    errors_total[i] = error

In [ ]:
x = LF_x
y = np.mean(LF_y, axis=0)
err = np.std(LF_y, axis=0)

In [ ]:
fig, ax = plt.subplots(figsize = (8,6))
ax.plot(x, y, '.', markersize=20)
ax.errorbar(x, y, yerr = err, fmt='none')
ax.set_yscale('log')
ax.set_ylabel(r'$\Phi$ [Mpc$^{-3}\Delta\log L^{-1}$]', size=15)
ax.set_xlabel(r'$\log_{10}(L_{\mathrm{Ly}\alpha}\,/\,\mathrm{erg}\,\mathrm{s}^{-1})$', size=15)
plt.show()

In [ ]:
weights = total_sel / n_lm
make_colorplot(cat, bb_ind, nb_ind, [], 'NB', False, weights)